In [ ]:
# Plot fft
fft_signal = np.fft.fft(filtered_signal)
fft_signal = np.abs(fft_signal) / len(filtered_signal)
freq = np.fft.fftfreq(len(signal), 1/fs)

plt.figure(figsize=(10, 5))
plt.plot(freq, np.abs(fft_signal))